In [ ]:
import pandas as pd

In [ ]:
# df = pd.read_csv("/ssdata/clinvar/clinvar/parseClinvar.csv")
df = pd.read_csv("/data/projects/processBio/clinvar/clinvar/parseClinvar.csv")

In [ ]:
df = df.assign(idx=df.index)

In [ ]:
df

In [ ]:
# sf = pd.read_csv("/ssdata/clinvar/clinvar/variant_summary.txt",delimiter="\t")
sf = pd.read_csv("/data/projects/processBio/clinvar/clinvar/variant_summary.txt",delimiter="\t")

In [ ]:
sf = sf.assign(nmid=sf.Name.apply(lambda s: s[:s.find("(")]))

In [ ]:
sf = sf.assign(variant=sf.Name.apply(lambda s: s[s.find("(p.") + 3 : -1]))

In [ ]:
sf

In [ ]:
ff = pd.merge(df,sf,how="inner",left_on=["NM","variant"], right_on=["nmid","variant"]).drop_duplicates(subset=["idx"])

In [ ]:
ff

In [ ]:
aaTable = {"Ala":"A",
           "Arg": "R",
           "Asn": "N",
           "Asp": "D",
           "Cys": "C",
           "Gln": "Q",
           "Glu": "E",
           "Gly": "G",
           "His": "H",
           "Ile": "I",
           "Leu": "L",
           "Lys": "K",
           "Met": "M",
           "Phe": "F",
           "Pro": "P",
           "Ser": "S",
           "Thr": "T",
           "Trp": "W",
           "Tyr": "Y",
           "Val": "V"}

In [ ]:
from processBioDBs.utilities import getSequence

In [ ]:
import pickle
seqs = pickle.load(open("/data/projects/processBio/clinvar/clinvar/seqs.pkl","rb"))

In [ ]:
symbols = set(ff.GeneSymbol)

symbols = [s for s in symbols if s not in seqs]

In [ ]:
skippedSymbols = set()

In [ ]:
from tqdm.notebook import tqdm
for s in tqdm(symbols):
    if s not in seqs:
        try:
            seqs[s] = getSequence(s)
        except:
            skippedSymbols.add(s)

In [ ]:
import numpy as np

In [ ]:
skippedVariants = set()

variants = []

for idx,row in tqdm(ff.iterrows(),total=ff.shape[0]):
    print(s)
    o,L,v = aaTable[row.variant[:3]], int(row.variant[3:-3]) - 1, aaTable[row.variant[-3:]]
    if row.GeneSymbol not in seqs or L >= len(seqs[row.GeneSymbol]) or seqs[row.GeneSymbol][L] != o:
        skippedVariants.add(idx)
        variants.append(np.nan)
    else:
        variant = s[:L] + v + s[L+1:]
        variants.append(variant)

In [ ]:
1 - len(skippedVariants)/ff.shape[0]

In [ ]:
ff = ff.assign(variantSeq=variants)

In [ ]:
ff

In [ ]:
ff.to_csv("/ssdata/clinvar/clinvar/parseClinvarWithVariant.csv")